In [3]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, Javascript
from modules import utility
import os
import pandas as pd
from medpy.metric.binary import hd, assd, dc, precision
import SimpleITK as sitk

def get_surface(img):
    a = sitk.GetImageFromArray(img)
    a = sitk.BinaryContour(a)
    a = sitk.GetArrayFromImage(a)
    return a.copy()

In [4]:
mhas = open('images.txt').readlines()
mhas = [i.replace('\n','') for i in mhas]

truths = open('truths.txt').readlines()
truths = [i.replace('\n','') for i in truths]

paths = open('paths.txt').readlines()
paths = [i.replace('\n','') for i in paths]

groups = open('groups.txt').readlines()
groups = [i.replace('\n','') for i in groups]

ct_test = open('ct_test.txt').readlines()
ct_test = [i.replace('\n','') for i in ct_test]

mr_test = open('mr_test.txt').readlines()
mr_test = [i.replace('\n','') for i in mr_test]

vtp_dir = '/home/marsdenlab/projects/DeepLofting/python/vtp/'
model_dir = '/home/marsdenlab/projects/DeepLofting/python/3d_models'

vtp_files = os.listdir(vtp_dir)

print ct_test
print vtp_files[:10]
IMAGES = ct_test+mr_test
print IMAGES
codes = ['i2ifc','i2i3d','i2i','drls','oof']
truth_files = [c for c in vtp_files if 'truth' in c]
print truth_files[:20]

test_images = []
for i in range(len(mhas)):
    if any([c in mhas[i] for c in IMAGES]):
        test_images.append((mhas[i],truths[i],paths[i],groups[i]))
        
#print test_images

['OSMSC0004', 'OSMSC0177', 'cabg11', 'OSMSC0179', 'OSMSC0144', 'OSMSC0148', 'OSMSC0150', 'OSMSC0119']
['0119_0001_i2ifc.LPA_br2_br1.vtp', '0150_0001_oof.celiac_hepatic.vtp', '0179_0001_truth.C_left.vtp', 'cabg11_all_oof.lca2.vtp', '0110_0001_i2i3d.right_iliac.vtp', '0177_0000_i2i3d.in_iliac_left.vtp', '0148_1001_i2i.renal_left.vtp', '0171_0001_i2i3d.R_incrd_b4.vtp', '0119_0001_i2i.RPA_br12_br4.vtp', 'cabg11_all_i2i.lsubcl2.vtp']
['OSMSC0004', 'OSMSC0177', 'cabg11', 'OSMSC0179', 'OSMSC0144', 'OSMSC0148', 'OSMSC0150', 'OSMSC0119', 'OSMSC0006', 'OSMSC0101', 'OSMSC0171', 'OSMSC0178', 'OSMSC0110', 'OSMSC0080', 'OSMSC0082']
['0179_0001_truth.C_left.vtp', '0179_0001_truth.R_ant.vtp', '0119_0001_truth.RPA_br8_br1.vtp', 'cabg11_all_truth.rsubcl3.vtp', '0119_0001_truth.RPA_br2.vtp', '0119_0001_truth.RPA_br9_br3.vtp', '0148_1001_truth.SMA.vtp', '0177_0000_truth.in_iliac_left.vtp', 'cabg11_all_truth.lsubcl3.vtp', '0171_0001_truth.L_incrd_b8.vtp', '0150_0001_truth.IMA.vtp', '0119_0001_truth.LPA_br4

In [5]:
import vtk
import SimpleITK as sitk
meta_dict = {}
for i in range(len(test_images)):
    im_file = test_images[i][0]
    print im_file
    reader = vtk.vtkMetaImageReader()
    reader.SetFileName(im_file)
    reader.Update()
    im = reader.GetOutput()

    spacing = im.GetSpacing()
    extent  = im.GetExtent()
    origin  = im.GetOrigin()
    s = extent[1::2]
    model = [c for c in IMAGES if c in im_file][0]
    model = model.replace('OSMSC','')
    meta_dict[model] = {}
    meta_dict[model]['spacing'] = spacing
    meta_dict[model]['origin'] = origin
    meta_dict[model]['extent'] = extent
print meta_dict.keys()

/home/marsdenlab/datasets/vascular_data/OSMSC0004/OSMSC0004-cm.mha
/home/marsdenlab/datasets/vascular_data/OSMSC0006/OSMSC0006-cm.mha
/home/marsdenlab/datasets/vascular_data/OSMSC0101/OSMSC0101-cm.mha
/home/marsdenlab/datasets/vascular_data/OSMSC0110/OSMSC0110-cm.mha
/home/marsdenlab/datasets/vascular_data/OSMSC0119/OSMSC0119-cm.mha
/home/marsdenlab/datasets/vascular_data/OSMSC0144/OSMSC0144-cm.mha
/home/marsdenlab/datasets/vascular_data/OSMSC0148/OSMSC0148-cm.mha
/home/marsdenlab/datasets/vascular_data/OSMSC0150/OSMSC0150-cm.mha
/home/marsdenlab/datasets/vascular_data/OSMSC0171/OSMSC0171-cm.mha
/home/marsdenlab/datasets/vascular_data/OSMSC0177/OSMSC0177-cm.mha
/home/marsdenlab/datasets/vascular_data/OSMSC0178/OSMSC0178-cm.mha
/home/marsdenlab/datasets/vascular_data/OSMSC0179/OSMSC0179-cm.mha
/home/marsdenlab/datasets/vascular_data/cabg11/cabg11-image.mha
['0004', '0178', '0006', '0179', '0171', '0119', '0101', '0110', '0177', '0144', '0150', 'cabg11', '0148']


In [ ]:
model_dict = {}
df = pd.DataFrame()
SKIP=3
for truth_fn in truth_files:
    print truth_fn
    truth_pd = utility.get_pds_from_files([vtp_dir+truth_fn])[0]
    
    image = truth_fn.split('_')[0]
    grp   = truth_fn.split('.')[1]
    spacing = meta_dict[image]['spacing']
    extent  = meta_dict[image]['extent'][1::2]
    origin  = meta_dict[image]['origin']
    
    truth_np = utility.pd_to_numpy_vol(truth_pd,spacing,
                                       shape=extent,origin=origin,foreground_value=1)[::SKIP,::SKIP,::SKIP]

    if np.sum(truth_np) < 1: continue
    truth_np = get_surface(truth_np.astype(int))
    for code in codes:
        code_fn = [c for c in vtp_files if (image in c) and (code+'.' in c) and (grp+'.' in c)]
        if len(code_fn) == 0: continue
        else:
            code_fn = code_fn[0]
            print code_fn
            
            code_pd = utility.get_pds_from_files([vtp_dir+code_fn])[0]
            
            
            code_np = utility.pd_to_numpy_vol(code_pd,spacing,shape=extent,
                                               origin=origin,foreground_value=1)[::SKIP,::SKIP,::SKIP]

            if np.sum(code_np) < 1: continue
            
            code_np = get_surface(code_np.astype(int))
            
            print truth_np.shape, code_np.shape, np.sum(truth_np),np.sum(code_np)
            
            e_hd = hd(truth_np,code_np, spacing[0])

#             e_j = utility.jaccard3D_pd_to_itk(model_dict[m]['truth']['pd'], model_dict[m][c]['pd'],im)

#             e_asd = assd(truth_np,pred_np, spacing[0])

#             e_dc = dc(truth_np,pred_np)

#             e_ap = precision(truth_np, pred_np)

#             d = {'image': m, 'code':c, 'jaccard':e_j, 'hausdorf':e_hd, 'ASSD':e_asd, 'dice':e_dc, 'ap':e_ap}
            d = {"image":image, "group":grp, 'code':code, 'hausdorf':e_hd}
            df = df.append(d, ignore_index=True)

0179_0001_truth.C_left.vtp
0179_0001_i2ifc.C_left.vtp
(89, 171, 171) (89, 171, 171) 2 65
0179_0001_i2i3d.C_left.vtp
(89, 171, 171) (89, 171, 171) 2 166
0179_0001_i2i.C_left.vtp
(89, 171, 171) (89, 171, 171) 2 123
0179_0001_drls.C_left.vtp
(89, 171, 171) (89, 171, 171) 2 261
0179_0001_oof.C_left.vtp
(89, 171, 171) (89, 171, 171) 2 373
0179_0001_truth.R_ant.vtp
0179_0001_i2ifc.R_ant.vtp
(89, 171, 171) (89, 171, 171) 48 707
0179_0001_i2i3d.R_ant.vtp
(89, 171, 171) (89, 171, 171) 48 469
0179_0001_i2i.R_ant.vtp
(89, 171, 171) (89, 171, 171) 48 603
0179_0001_drls.R_ant.vtp
(89, 171, 171) (89, 171, 171) 48 784
0179_0001_oof.R_ant.vtp
(89, 171, 171) (89, 171, 171) 48 1068
0119_0001_truth.RPA_br8_br1.vtp
0119_0001_i2ifc.RPA_br8_br1.vtp
(161, 171, 171) (161, 171, 171) 18 59
0119_0001_i2i3d.RPA_br8_br1.vtp
(161, 171, 171) (161, 171, 171) 18 17
0119_0001_i2i.RPA_br8_br1.vtp
(161, 171, 171) (161, 171, 171) 18 54
0119_0001_drls.RPA_br8_br1.vtp
(161, 171, 171) (161, 171, 171) 18 135
0119_0001_oof.RPA

(303, 171, 171) (303, 171, 171) 4748 4470
0177_0000_drls.disection.vtp
(303, 171, 171) (303, 171, 171) 4748 2411
0177_0000_oof.disection.vtp
(303, 171, 171) (303, 171, 171) 4748 3343
0119_0001_truth.LPA_br6.vtp
0119_0001_i2ifc.LPA_br6.vtp
(161, 171, 171) (161, 171, 171) 48 115
0119_0001_i2i3d.LPA_br6.vtp
(161, 171, 171) (161, 171, 171) 48 103
0119_0001_i2i.LPA_br6.vtp
(161, 171, 171) (161, 171, 171) 48 112
0119_0001_drls.LPA_br6.vtp
(161, 171, 171) (161, 171, 171) 48 589
0119_0001_oof.LPA_br6.vtp
(161, 171, 171) (161, 171, 171) 48 694
0171_0001_truth.L_incrd_b6.vtp
0171_0001_i2ifc.L_incrd_b6.vtp
(48, 171, 171) (48, 171, 171) 8 49
0171_0001_i2i3d.L_incrd_b6.vtp
(48, 171, 171) (48, 171, 171) 8 67
0171_0001_i2i.L_incrd_b6.vtp
(48, 171, 171) (48, 171, 171) 8 23
0171_0001_drls.L_incrd_b6.vtp
(48, 171, 171) (48, 171, 171) 8 1326
0171_0001_oof.L_incrd_b6.vtp
(48, 171, 171) (48, 171, 171) 8 839
0148_1001_truth.IMA.vtp
0148_1001_i2ifc.IMA.vtp
(50, 171, 171) (50, 171, 171) 6 117
0148_1001_i2i3d.

0179_0001_drls.L_vtbrl.vtp
(89, 171, 171) (89, 171, 171) 527 3416
0179_0001_oof.L_vtbrl.vtp
(89, 171, 171) (89, 171, 171) 527 4190
0110_0001_truth.right_iliac_branch2.vtp
0110_0001_i2ifc.right_iliac_branch2.vtp
(171, 21, 171) (171, 21, 171) 10 19
0110_0001_i2i3d.right_iliac_branch2.vtp
(171, 21, 171) (171, 21, 171) 10 34
0110_0001_i2i.right_iliac_branch2.vtp
(171, 21, 171) (171, 21, 171) 10 31
0110_0001_drls.right_iliac_branch2.vtp
(171, 21, 171) (171, 21, 171) 10 75
0110_0001_oof.right_iliac_branch2.vtp
(171, 21, 171) (171, 21, 171) 10 33
0119_0001_truth.LPA_br1_br1_br4.vtp
0119_0001_i2ifc.LPA_br1_br1_br4.vtp
(161, 171, 171) (161, 171, 171) 23 55
0119_0001_i2i3d.LPA_br1_br1_br4.vtp
(161, 171, 171) (161, 171, 171) 23 12
0119_0001_i2i.LPA_br1_br1_br4.vtp
(161, 171, 171) (161, 171, 171) 23 53
0119_0001_drls.LPA_br1_br1_br4.vtp
(161, 171, 171) (161, 171, 171) 23 204
0119_0001_oof.LPA_br1_br1_br4.vtp
(161, 171, 171) (161, 171, 171) 23 458
0171_0001_truth.L_antrb_b2.vtp
0171_0001_i2ifc.L_an

0110_0001_i2ifc.left_sup_renal.vtp
(171, 21, 171) (171, 21, 171) 40 118
0110_0001_i2i3d.left_sup_renal.vtp
(171, 21, 171) (171, 21, 171) 40 89
0110_0001_i2i.left_sup_renal.vtp
(171, 21, 171) (171, 21, 171) 40 103
0110_0001_drls.left_sup_renal.vtp
(171, 21, 171) (171, 21, 171) 40 315
0110_0001_oof.left_sup_renal.vtp
(171, 21, 171) (171, 21, 171) 40 195
0171_0001_truth.L_incrd_b10.vtp
0171_0001_i2ifc.L_incrd_b10.vtp
(48, 171, 171) (48, 171, 171) 2 276
0171_0001_i2i3d.L_incrd_b10.vtp
(48, 171, 171) (48, 171, 171) 2 13
0171_0001_i2i.L_incrd_b10.vtp
(48, 171, 171) (48, 171, 171) 2 1007
0171_0001_drls.L_incrd_b10.vtp
(48, 171, 171) (48, 171, 171) 2 1040
0171_0001_oof.L_incrd_b10.vtp
(48, 171, 171) (48, 171, 171) 2 629
0177_0000_truth.IMA.vtp
0177_0000_i2ifc.IMA.vtp
(303, 171, 171) (303, 171, 171) 40 147
0177_0000_i2i3d.IMA.vtp
(303, 171, 171) (303, 171, 171) 40 538
0177_0000_i2i.IMA.vtp
(303, 171, 171) (303, 171, 171) 40 261
0177_0000_drls.IMA.vtp
(303, 171, 171) (303, 171, 171) 40 287
0177_

In [51]:
strings = []
ct_test = [c.replace('OSMSC','') for c in ct_test]
mr_test = [c.replace('OSMSC','') for c in mr_test]
print mr_test
for CODE in ['CT','MR']:
    l = ct_test
    if CODE == 'MR': l = mr_test
    strings = []
    f = open('{}_3d_err_.txt'.format(CODE),'w')
    for c in codes:
        print c
        d = df.loc[df['code']==c]
        t = [any([H==K for H in l]) for K in d['image']]

        d = d.loc[t]
        print d.shape
        a = d.mean()
        
        s = "{} & ${:.3f}$$ \\".format(c,a['hausdorf'])
        
        strings.append((a['hausdorf'],s))

    strings = sorted(strings,key = lambda x: x[0])
    print CODE
    for s in strings:
        print s[1]
        f.write(s[1]+'\n')
    f.close()

['0006', '0101', '0171', '0178', '0110']
i2ifc
(211, 4)
i2i3d
(193, 4)
i2i
(210, 4)
drls
(211, 4)
oof
(211, 4)
CT
i2ifc & $0.542$$ \
i2i & $0.576$$ \
drls & $0.641$$ \
oof & $0.662$$ \
i2i3d & $0.726$$ \
i2ifc
(90, 4)
i2i3d
(88, 4)
i2i
(90, 4)
drls
(90, 4)
oof
(90, 4)
MR
i2i & $0.366$$ \
i2ifc & $0.373$$ \
i2i3d & $0.513$$ \
oof & $0.523$$ \
drls & $0.581$$ \


In [42]:
df.to_csv('3derrs.csv')